In [1]:
import ollama

In [8]:
import ollama
from typing import List, Dict, Optional

from pydantic import BaseModel, Field
from typing import Literal, Dict, Any

class FunctionParameters(BaseModel):
    type: Literal["object"] = "object"
    properties: Dict[str, Dict[str, Any]]
    required: list[str]

class Function(BaseModel):
    name: str = Field(description="Name of the function to call")
    description: str = Field(description="Description of what the function does")
    parameters: FunctionParameters

class OllamaTool(BaseModel):
    type: Literal["function"] = "function"
    function: Function

# Example usage for search function
search_parameters = FunctionParameters(
    properties={
        "query": {
            "type": "string",
            "description": "The search query."
        }
    },
    required=["query"]
)

search_function = Function(
    name="get_search_results",
    description="Get search results for a provided query.",
    parameters=search_parameters
)

tools_schema = [OllamaTool(
    function=search_function
).model_dump()]

def ollama_function_selection(user_message: str,
                            tools_schema: List,
                            model_name: str = "llama3.1:8b") -> Optional[dict]:
    """
    Get function calling arguments from Ollama model for a single function call.
    Returns None if no function is selected.
    """
    messages=[{
        "role": "user", 
        "content": user_message
    }]
    
    response = ollama.chat(
        model=model_name,
        messages=messages,
        tools=tools_schema
    )

    if not response["message"].get("tool_calls"):
        return None
    
    # Return arguments of first tool call
    tool = response["message"]["tool_calls"][0]
    return tool["function"]["arguments"]

In [9]:
ollama_function_selection(
    "How does HNSW work?",
    tools_schema=tools_schema,
    model_name="llama3.1:8b"
)

{'query': 'HNSW algorithm'}

In [10]:
ollama_function_selection(
    "How does HNSW work?",
    tools_schema=tools_schema,
    model_name="llama3.2:3b"
)

{'query': 'HNSW how it works'}

In [14]:
import ollama
from typing import List, Dict, Optional

from pydantic import BaseModel, Field
from typing import Literal, Dict, Any

class FunctionParameters(BaseModel):
    type: Literal["object"] = "object"
    properties: Dict[str, Dict[str, Any]]
    required: list[str]

class Function(BaseModel):
    name: str = Field(description="Name of the function to call")
    description: str = Field(description="Description of what the function does")
    parameters: FunctionParameters

class OllamaTool(BaseModel):
    type: Literal["function"] = "function"
    function: Function

class SearchQuery(BaseModel):
    query: str = Field(description="The search query to execute")
    target_collection: str = Field(description="The collection to search in", default="default")
    filters: str = Field(description="Any filters to apply to the search", default="")

# Get the schema for SearchQuery, which defines the query parameter structure
search_query_schema = SearchQuery.model_json_schema()
query_properties = search_query_schema["properties"]

# Example usage for search function
search_parameters = FunctionParameters(
    properties={
        "query": query_properties["query"],
        "target_collection": query_properties["target_collection"],
        "filters": query_properties["filters"]
    },
    required=["query", "target_collection"]  # Make both fields required
)

search_function = Function(
    name="get_search_results",
    description="Search for documents in a specified collection using a query and optional filters.",
    parameters=search_parameters
)

tools_schema = [OllamaTool(
    function=search_function
).model_dump()]

def ollama_function_selection(user_message: str,
                            tools_schema: List,
                            model_name: str = "llama3.1:8b") -> Optional[dict]:
    """
    Get function calling arguments from Ollama model for a single function call.
    Returns None if no function is selected.
    """
    messages=[{
        "role": "system",
        "content": "You are a helpful assistant that searches through document collections. When given a query, you should specify both the search query and target collection."
    }, {
        "role": "user", 
        "content": user_message
    }]
    
    response = ollama.chat(
        model=model_name,
        messages=messages,
        tools=tools_schema
    )

    if not response["message"].get("tool_calls"):
        return None
    
    # Return arguments of first tool call
    tool = response["message"]["tool_calls"][0]
    return tool["function"]["arguments"]

In [18]:
model_response = ollama_function_selection(
    "How does HNSW work?",
    tools_schema=tools_schema,
    model_name="llama3.1:8b"
)

print(model_response)

{'filters': '', 'query': 'HNSW algorithm', 'target_collection': 'Computer Science Documentation'}


## `WeaviateQuery` model

```python
class IntPropertyFilter(BaseModel):
    property_name: str
    operator: Literal["=", "<", ">", "<=", ">="]
    value: int | float

class TextPropertyFilter(BaseModel):
    property_name: str
    operator: Literal["=", "LIKE"]
    value: str

class BooleanPropertyFilter(BaseModel):
    property_name: str
    operator: Literal["=", "!="]
    value: bool

class IntAggregation(BaseModel):
    property_name: str
    metrics: Literal["COUNT", "TYPE", "MIN", "MAX", "MEAN", "MEDIAN", "MODE", "SUM"]

class TextAggregation(BaseModel):
    property_name: str
    metrics: Literal["COUNT", "TYPE", "TOP_OCCURRENCES"]
    top_occurrences_limit: Optional[int] = None

class BooleanAggregation(BaseModel):
    property_name: str
    metrics: Literal["COUNT", "TYPE", "TOTAL_TRUE", "TOTAL_FALSE", "PERCENTAGE_TRUE", "PERCENTAGE_FALSE"]

class WeaviateQuery(BaseModel):
    corresponding_natural_language_query: str
    target_collection: str
    search_query: Optional[str]
    integer_property_filter: Optional[IntPropertyFilter]
    text_property_filter: Optional[TextPropertyFilter]
    boolean_property_filter: Optional[BooleanPropertyFilter]
    integer_property_aggregation: Optional[IntAggregation]
    text_property_aggregation: Optional[TextAggregation]
    boolean_property_aggregation: Optional[BooleanAggregation]
    groupby_property: Optional[str]
```


In [53]:
import ollama
from typing import List, Dict, Optional, Union
from pydantic import BaseModel, Field
from typing import Literal, Dict, Any

# Import the WeaviateQuery model and its dependencies from above
class IntPropertyFilter(BaseModel):
    property_name: str
    operator: Literal["=", "<", ">", "<=", ">="]
    value: int | float

class TextPropertyFilter(BaseModel):
    property_name: str
    operator: Literal["=", "LIKE"]
    value: str

class BooleanPropertyFilter(BaseModel):
    property_name: str
    operator: Literal["=", "!="]
    value: bool

class IntAggregation(BaseModel):
    property_name: str
    metrics: Literal["COUNT", "TYPE", "MIN", "MAX", "MEAN", "MEDIAN", "MODE", "SUM"]

class TextAggregation(BaseModel):
    property_name: str
    metrics: Literal["COUNT", "TYPE", "TOP_OCCURRENCES"]
    top_occurrences_limit: Optional[int] = None

class BooleanAggregation(BaseModel):
    property_name: str
    metrics: Literal["COUNT", "TYPE", "TOTAL_TRUE", "TOTAL_FALSE", "PERCENTAGE_TRUE", "PERCENTAGE_FALSE"]

class WeaviateQuery(BaseModel):
    corresponding_natural_language_query: str
    target_collection: str
    search_query: Optional[str]
    integer_property_filter: Optional[IntPropertyFilter]
    text_property_filter: Optional[TextPropertyFilter]
    boolean_property_filter: Optional[BooleanPropertyFilter]
    integer_property_aggregation: Optional[IntAggregation]
    text_property_aggregation: Optional[TextAggregation]
    boolean_property_aggregation: Optional[BooleanAggregation]
    groupby_property: Optional[str]

class OllamaFunction(BaseModel):
    name: str
    description: str
    parameters: dict

class OllamaTool(BaseModel):
    type: Literal["function"] = "function"
    function: OllamaFunction

# Get the schema for WeaviateQuery
weaviate_query_schema = WeaviateQuery.model_json_schema()
query_properties = weaviate_query_schema["properties"]

# Define function parameters using WeaviateQuery schema
search_parameters = {
    "type": "object",
    "properties": {
        "corresponding_natural_language_query": query_properties["corresponding_natural_language_query"],
        "target_collection": query_properties["target_collection"],
        "search_query": query_properties["search_query"],
        "integer_property_filter": {
            "type": "object",
            "properties": {
                "property_name": {"type": "string"},
                "operator": {"type": "string", "enum": ["=", "<", ">", "<=", ">="]},
                "value": {"type": "number"}
            },
            "required": ["property_name", "operator", "value"]
        },
        "text_property_filter": {
            "type": "object", 
            "properties": {
                "property_name": {"type": "string"},
                "operator": {"type": "string", "enum": ["=", "LIKE"]},
                "value": {"type": "string"}
            },
            "required": ["property_name", "operator", "value"]
        },
        "boolean_property_filter": {
            "type": "object",
            "properties": {
                "property_name": {"type": "string"},
                "operator": {"type": "string", "enum": ["=", "!="]},
                "value": {"type": "boolean"}
            },
            "required": ["property_name", "operator", "value"]
        },
        "integer_property_aggregation": {
            "type": "object",
            "properties": {
                "property_name": {"type": "string"},
                "metrics": {"type": "string", "enum": ["COUNT", "TYPE", "MIN", "MAX", "MEAN", "MEDIAN", "MODE", "SUM"]}
            },
            "required": ["property_name", "metrics"]
        },
        "text_property_aggregation": {
            "type": "object",
            "properties": {
                "property_name": {"type": "string"},
                "metrics": {"type": "string", "enum": ["COUNT", "TYPE", "TOP_OCCURRENCES"]},
                "top_occurrences_limit": {"type": "integer"}
            },
            "required": ["property_name", "metrics"]
        },
        "boolean_property_aggregation": {
            "type": "object",
            "properties": {
                "property_name": {"type": "string"},
                "metrics": {"type": "string", "enum": ["COUNT", "TYPE", "TOTAL_TRUE", "TOTAL_FALSE", "PERCENTAGE_TRUE", "PERCENTAGE_FALSE"]}
            },
            "required": ["property_name", "metrics"]
        },
        "groupby_property": query_properties["groupby_property"]
    },
    "required": ["corresponding_natural_language_query", "target_collection"]
}

tools_schema = [OllamaTool(
    type="function",
    function=OllamaFunction(
        name="get_search_results",
        description="""Search for documents in a specified collection using a query and optional filters and aggregations.
        When filtering by dates or years, use integer_property_filter with the appropriate property name and operator.""",
        parameters=search_parameters
    )
).model_dump()]

def ollama_function_selection(user_message: str,
                            tools_schema: List,
                            model_name: str = "llama3.1:8b") -> Optional[dict]:
    """
    Get function calling arguments from Ollama model for a single function call.
    Returns None if no function is selected.
    """
    messages=[{
        "role": "system",
        "content": """You are a helpful assistant that searches through document collections. When given a query, you should specify both the search query and target collection.
        When filtering by dates or years, use integer_property_filter with the correct operator format, e.g.:
        {"property_name": "published_year", "operator": ">=", "value": 2022}"""
    }, {
        "role": "user", 
        "content": user_message
    }]
    
    response = ollama.chat(
        model=model_name,
        messages=messages,
        tools=tools_schema
    )

    if not response["message"].get("tool_calls"):
        return None
    
    # Return arguments of first tool call
    tool = response["message"]["tool_calls"][0]
    return tool["function"]["arguments"]

In [54]:
model_response = ollama_function_selection(
    "How does HNSW work?",
    tools_schema=tools_schema,
    model_name="llama3.1:8b"
)

print(model_response)

{'corresponding_natural_language_query': 'How does HNSW work?', 'target_collection': 'knowledge_base'}


In [40]:
model_response = ollama_function_selection(
    "How does HNSW work? Filters by articles at least as new as 2022",
    tools_schema=tools_schema,
    model_name="llama3.1:8b"
)

print(model_response)

{'corresponding_natural_language_query': 'How does HNSW work?', 'integer_property_filter': "{'property_name': 'published_year', 'operator': '>=', 'value': 2022}", 'target_collection': 'articles'}


In [47]:
type(model_response["integer_property_filter"])

str

In [50]:
import json
from typing import Dict, Any

def parse_int_property_filter(model_response: Dict[str, Any]) -> IntPropertyFilter:
    # Replace single quotes with double quotes to make it valid JSON
    json_str = model_response["integer_property_filter"].replace("'", '"')
    filter_dict = json.loads(json_str)
    return IntPropertyFilter(
        property_name=filter_dict["property_name"],
        operator=filter_dict["operator"],
        value=filter_dict["value"]
    )

parse_int_property_filter(model_response)

IntPropertyFilter(property_name='published_year', operator='>=', value=2022)

# Limitation of Filter Models

In [52]:
model_response = ollama_function_selection(
    "How does HNSW work? Find articles from 2022 or later that have more than 1000 citations",
    tools_schema=tools_schema,
    model_name="llama3.1:8b"
)

print(model_response)

{'corresponding_natural_language_query': 'How does HNSW work?', 'integer_property_filter': {'operator': '>=', 'property_name': 'published_year', 'value': 2022}, 'target_collection': 'articles'}
